In [24]:
"""The purpose of this notebook is to write a .py callable program which does five things: 1) create a word2vec neural network 2) optimize the parameters for
NN creation 3) save the NN 4) graph the data, and 5) make the word embeddings compatible with the embeddings created by BERT. Not neccessarily in that order.
It should take in the locationof a data corpus, the desired name of the saved NN"""

#need to double check if we need these if they're already in Lousia's functions
import re, string 
import pandas as pd 
from time import time  
from collections import defaultdict
import spacy
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import matplotlib.pyplot as plt
import json
%matplotlib inline
import torch
from transformers import BertModel, BertConfig, BertTokenizer, PreTrainedTokenizer
import csv

#louisa's functions imported from local download as .py, added lines = true to fix value error when reading json into df
import Louisa_w2v_functions as w2v_functions



In [26]:
#write a quick function that prompts the user for a local corpus folder name then reads all the jsons in it, they then can be all cleaned w/ Louisa's clean
#function or a one I wrote

#w2v_functions.feed2vec(r"C:\Users\bowri\square1\ChemLP\Bowman\textbook_files\Brewing_Science_A_Multidisciplinary_Approach_by_Mosher_and_Trantham.json", tokenize = True )


"""this function prompts the user for a local folder/corpus location, where all the json's are located. It then reads all of them into a list"""



data init is  <class 'list'>


Keyword arguments {'add_special_tokens': True} not recognized.


[['0', 'michael', 'moshe', '##r', '·', 'kenneth', 'tran', '##tham', 'brewing', 'sci', '##e', '.', '.', '.', '1', 'rights', 'reserved', 'publisher', 'whether', 'whole', 'part', 'm', '.', '.', '.', '2', 'use', 'general', 'descriptive', 'names', 'registered', 'names', '.', '.', '.', '3', 'publisher', 'authors', 'editors', 'safe', 'assume', 'advice', 'i', '.', '.', '.', '4', 'neither', 'publisher', 'authors', 'editors', 'give', 'warrant', '.', '.', '.', '.', '.', '.', '131', '##7', '##6', '131', '##7', '##7', 'k', 'g', 'k', 'j', 'k', 'p', 'r', 'n', 'e', 'g', 'k', 'j', 'k', 'p', 'l', 'h', 'n', 'e', 'g', 'k', 'j', 'k', 'g', '.', '.', '.', '131', '##7', '##8', 'b', 'e', 'l', 'b', 'appendix', 'b', 'ref', '##ri', '##ger', '##ant', 'data', 'table', 'su', '##pe', '.', '.', '.', '131', '##7', '##9', 'moshe', '##r', 'k', '131', '##80', 'tran', '##tham', 'brewing', 'science', 'multi', '##dis', '##ci', '##plin', '##ary', 'app', '.', '.', '.', 'name', ':', 'text', ',', 'length', ':', '81', '##7', '##9

In [28]:
"""Not entirely sure what to do here; this function needs to read all of the local textbooks and build a word2vec vocab. The debate I'm having is whether
or not it would be faster to train 1 model on a super-corpus of all the books, or keep updating the vocabulary w new books and training it each time. The first 
seems better to me. But it's going to produce a word2vec model w a built vocab that can then be trained iteratively until certain parameters are optimized"""

#def vocab_builder

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-28-08588e0fd2f2>, line 2)

In [29]:
"""This function is going to iteratively train the word2vec model until certain parameters are optimized. It's going to need grip search/sci-kit?""""

#def trainer_optimizer

SyntaxError: EOL while scanning string literal (<ipython-input-29-aa178858a1e8>, line 1)

In [30]:
"""This function/part can handle the TSNE graphing, display to user and such"""

#def grapher

'This function/part can handle the TSNE graphing, display to user and such'

In [31]:
"""This function will make the word2vec vectors compatible w/ the BERT vectors."""

#def vector_changer

'This function will make the word2vec vectors compatible w/ the BERT vectors.'

In [ ]:
"""Main part of the program here. Call all the functions? 